In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = ""

: 

In [ ]:
from fedcore.tools.example_utils import get_scenario_for_api
from fedcore.api.main import FedCore
from fedcore.api.utils.checkers_collection import ApiConfigCheck
from fedcore.data.dataloader import load_data
from fedcore.api.utils.evaluation import evaluate_original_model, evaluate_optimised_model
from fedcore.repository.config_repository import DEFAULT_CLF_API_CONFIG
from torchvision.models import resnet18, ResNet18_Weights

from fedcore.api.config_factory import ConfigFactory
from fedcore.api.api_configs import (
    APIConfigTemplate, AutoMLConfigTemplate,
    LearningConfigTemplate, FedotConfigTemplate,
    LowRankTemplate)

initial_assumption = resnet18(ResNet18_Weights)

DATASET = 'CIFAR10'
DATASET_PARAMS = {'train_bs': 64,
                  'val_bs': 100,
                  'train_shuffle': True,
                  'val_shuffle': False}

METRIC_TO_OPTIMISE = ['accuracy', 'latency', 'throughput']


APIConfig = ConfigFactory.from_template(
    APIConfigTemplate(
        None,
        AutoMLConfigTemplate(
            FedotConfigTemplate(
            problem='classification',
            pop_size=1,
            timeout=1,
            initial_assumption=initial_assumption
            )
        ),
        LearningConfigTemplate(
            criterion='cross_entropy',
            learning_strategy='from_scratch',
            peft_strategy='low_rank',
            peft_strategy_params=LowRankTemplate(
                epochs=1,
                non_adaptive_threshold=0.1
            ),
        )
    )
)

api_config = APIConfig() # Here we can update with kwargs

In [ ]:
input_data = load_data(DATASET)
fedcore_compressor = FedCore(api_config)
fedcore_compressor.fit_no_evo(input_data);